<a href="https://colab.research.google.com/github/markmaloba/DSCORE-Week-6-Independent-Project/blob/master/DSCore_W6_IP_Mark_Maloba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Load all necessary libraries & prerequisites

import numpy as np
import pandas as pd
import seaborn as sb
import scipy as sp

from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
ranking = pd.read_csv('fifa_ranking.csv')
results = pd.read_csv('results.csv')

In [27]:
ranking.head(10)

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
5,6,Republic of Ireland,IRL,0.0,54,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
6,7,Russia,RUS,0.0,52,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
7,8,Brazil,BRA,0.0,55,-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
8,9,Norway,NOR,0.0,49,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
9,10,Denmark,DEN,0.0,51,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08


In [5]:
results.head(10)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
5,1876-03-25,Scotland,Wales,4,0,Friendly,Glasgow,Scotland,False
6,1877-03-03,England,Scotland,1,3,Friendly,London,England,False
7,1877-03-05,Wales,Scotland,0,2,Friendly,Wrexham,Wales,False
8,1878-03-02,Scotland,England,7,2,Friendly,Glasgow,Scotland,False
9,1878-03-23,Scotland,Wales,9,0,Friendly,Glasgow,Scotland,False


For our polynomial analysis, the following data is required for training:
Rank of home team

1. Home team
2. Away team
3. Home team score
4. Away team score
5. Rank of home team
6. Rank of away team
7. Tournament type

Everything else can be discarded for the purposes of our test


In [8]:
#Check columns in both tables to better assess missing parts in each data set, and merging possibilities
print(results.columns)
print(ranking.columns)

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'city', 'country', 'neutral'],
      dtype='object')
Index(['rank', 'country_full', 'country_abrv', 'total_points',
       'previous_points', 'rank_change', 'cur_year_avg',
       'cur_year_avg_weighted', 'last_year_avg', 'last_year_avg_weighted',
       'two_year_ago_avg', 'two_year_ago_weighted', 'three_year_ago_avg',
       'three_year_ago_weighted', 'confederation', 'rank_date'],
      dtype='object')


# Merging & Cleaning

In [9]:
#Merging the 2 datasets

merge_df = results.merge(ranking, left_on = ['date', 'home_team'],
                        right_on = ['rank_date', 'country_full'], how = 'inner')
merge_df.head(10)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1993-08-08,Bolivia,Uruguay,3,1,FIFA World Cup qualification,La Paz,Bolivia,False,59,Bolivia,BOL,0.0,13,28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
1,1993-08-08,Brazil,Mexico,1,1,Friendly,Maceió,Brazil,False,8,Brazil,BRA,0.0,55,-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
2,1993-08-08,Ecuador,Venezuela,5,0,FIFA World Cup qualification,Quito,Ecuador,False,35,Ecuador,ECU,0.0,23,30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
3,1993-08-08,Guinea,Sierra Leone,1,0,Friendly,Conakry,Guinea,False,65,Guinea,GUI,0.0,23,-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,1993-08-08
4,1993-08-08,Paraguay,Argentina,1,3,FIFA World Cup qualification,Asunción,Paraguay,False,67,Paraguay,PAR,0.0,22,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
5,1993-08-08,Peru,Colombia,0,1,FIFA World Cup qualification,Lima,Peru,False,70,Peru,PER,0.0,16,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
6,1993-08-08,Zimbabwe,Eswatini,2,0,Friendly,Harare,Zimbabwe,False,50,Zimbabwe,ZIM,0.0,27,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,1993-08-08
7,1993-09-23,Saudi Arabia,Costa Rica,1,2,Friendly,Khobar,Saudi Arabia,False,44,Saudi Arabia,KSA,0.0,34,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AFC,1993-09-23
8,1994-03-15,Gabon,DR Congo,2,1,Friendly,Libreville,Gabon,False,61,Gabon,GAB,0.0,29,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,1994-03-15
9,1994-03-15,Israel,Ukraine,1,0,Friendly,Haifa,Israel,False,54,Israel,ISR,0.0,30,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1994-03-15


In [13]:
# Drop unnecessary columns
# Rename rank column
merge_df.rename(columns = {'rank' : 'home_rank'}, inplace = True)
print(merge_df.columns)

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'city', 'country', 'neutral', 'home_rank', 'country_full',
       'country_abrv', 'total_points', 'previous_points', 'rank_change',
       'cur_year_avg', 'cur_year_avg_weighted', 'last_year_avg',
       'last_year_avg_weighted', 'two_year_ago_avg', 'two_year_ago_weighted',
       'three_year_ago_avg', 'three_year_ago_weighted', 'confederation',
       'rank_date'],
      dtype='object')


In [0]:
# Drop unnecessary columns
merge_df.drop(columns = ['country_abrv', 'total_points', 'previous_points', 'rank_change', 'city', 'rank_date', 'confederation'
                        'cur_year_avg', 'cur_year_avg_weighted', 'last_year_avg', 'last_year_avg_weighted',
                        'two_year_ago_avg', 'two_year_ago_weighted', 'three_year_ago_avg', 
                        'three_year_ago_weighted'], axis = 1, inplace = True)

Let's see how manageable our dataset is now

In [16]:
merge_df.head(7)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,home_rank,country_full,confederation,rank_date
0,1993-08-08,Bolivia,Uruguay,3,1,FIFA World Cup qualification,La Paz,Bolivia,False,59,Bolivia,CONMEBOL,1993-08-08
1,1993-08-08,Brazil,Mexico,1,1,Friendly,Maceió,Brazil,False,8,Brazil,CONMEBOL,1993-08-08
2,1993-08-08,Ecuador,Venezuela,5,0,FIFA World Cup qualification,Quito,Ecuador,False,35,Ecuador,CONMEBOL,1993-08-08
3,1993-08-08,Guinea,Sierra Leone,1,0,Friendly,Conakry,Guinea,False,65,Guinea,CAF,1993-08-08
4,1993-08-08,Paraguay,Argentina,1,3,FIFA World Cup qualification,Asunción,Paraguay,False,67,Paraguay,CONMEBOL,1993-08-08
5,1993-08-08,Peru,Colombia,0,1,FIFA World Cup qualification,Lima,Peru,False,70,Peru,CONMEBOL,1993-08-08
6,1993-08-08,Zimbabwe,Eswatini,2,0,Friendly,Harare,Zimbabwe,False,50,Zimbabwe,CAF,1993-08-08


In [0]:
merge_df.drop(columns = ['city', 'rank_date', 'confederation'], axis = 1, inplace = True)

In [0]:
merge_df.drop(columns = ['date'], axis = 1, inplace = True)

In [24]:
merge_df.head(7)

,home_team,away_team,home_score,away_score,tournament,country,neutral,home_rank,country_full
0,Bolivia,Uruguay,3,1,FIFA World Cup qualification,Bolivia,False,59,Bolivia
1,Brazil,Mexico,1,1,Friendly,Brazil,False,8,Brazil
2,Ecuador,Venezuela,5,0,FIFA World Cup qualification,Ecuador,False,35,Ecuador
3,Guinea,Sierra Leone,1,0,Friendly,Guinea,False,65,Guinea
4,Paraguay,Argentina,1,3,FIFA World Cup qualification,Paraguay,False,67,Paraguay
5,Peru,Colombia,0,1,FIFA World Cup qualification,Peru,False,70,Peru
6,Zimbabwe,Eswatini,2,0,Friendly,Zimbabwe,False,50,Zimbabwe


In [56]:
# Check for null values and drop them if necessary
merge_df.isnull().any()

home_team       False
away_team       False
home_score      False
away_score      False
tournament      False
country         False
neutral         False
home_rank       False
country_full    False
dtype: bool

**RESULT**: No nulll values

# Feature engineering

We can merge both tables to get the data we need, then create a few custom columns:
1. Turn the tournament type into categorical data
2. Create home team rank, and away team rank columns

In [25]:
merge_df.tournament.unique()

array(['FIFA World Cup qualification', 'Friendly', 'CFU Caribbean Cup',
       'Merdeka Tournament', 'South Pacific Games',
       'African Cup of Nations', 'AFC Asian Cup qualification',
       'CFU Caribbean Cup qualification', 'Copa América',
       'UEFA Euro qualification',
       'United Arab Emirates Friendship Tournament', 'UNCAF Cup',
       'Korea Cup', 'Gold Cup', 'USA Cup', 'Millennium Cup', "King's Cup",
       'Windward Islands Tournament', 'Gulf Cup',
       'Cyprus International Tournament',
       'Malta International Tournament', 'Gold Cup qualification',
       'UAFA Cup', 'AFF Championship', 'SAFF Cup', "Prime Minister's Cup",
       'Confederations Cup', 'AFC Asian Cup', 'Nehru Cup', 'COSAFA Cup',
       'CECAFA Cup', 'EAFF Championship',
       'AFC Challenge Cup qualification',
       'African Cup of Nations qualification', 'Copa del Pacífico',
       'Oceania Nations Cup', 'African Nations Championship',
       'Intercontinental Cup'], dtype=object)

In [0]:
tourn_types=[merge_df.tournament.unique()]

In [52]:
# Output unique items from a column in to an array.
# In this case the unique items are the different tournament types

typesss = merge_df.tournament.unique()
qq = []
for x in typesss:
    if x not in qq:
        qq.append(x)
print(qq)

['FIFA World Cup qualification', 'Friendly', 'CFU Caribbean Cup', 'Merdeka Tournament', 'South Pacific Games', 'African Cup of Nations', 'AFC Asian Cup qualification', 'CFU Caribbean Cup qualification', 'Copa América', 'UEFA Euro qualification', 'United Arab Emirates Friendship Tournament', 'UNCAF Cup', 'Korea Cup', 'Gold Cup', 'USA Cup', 'Millennium Cup', "King's Cup", 'Windward Islands Tournament', 'Gulf Cup', 'Cyprus International Tournament', 'Malta International Tournament', 'Gold Cup qualification', 'UAFA Cup', 'AFF Championship', 'SAFF Cup', "Prime Minister's Cup", 'Confederations Cup', 'AFC Asian Cup', 'Nehru Cup', 'COSAFA Cup', 'CECAFA Cup', 'EAFF Championship', 'AFC Challenge Cup qualification', 'African Cup of Nations qualification', 'Copa del Pacífico', 'Oceania Nations Cup', 'African Nations Championship', 'Intercontinental Cup']


In [0]:
# This section I have used to generate random integers corresponding with each type of tournament

d = {ni: indi for indi, ni in enumerate(set(qq))}
tc = [d[ni] for ni in qq]

In [49]:
print (tc)

[11, 17, 8, 37, 5, 9, 34, 1, 26, 29, 18, 28, 10, 15, 33, 22, 20, 19, 35, 6, 32, 30, 36, 23, 25, 14, 21, 27, 4, 7, 0, 31, 16, 13, 12, 24, 2, 3]


# EDA

In [57]:
import pandas_profiling as pp
pp.ProfileReport(merge_df)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,9
Number of observations,957
Total Missing (%),0.0%
Total size in memory,68.2 KiB
Average record size in memory,73.0 B
Numeric,3
Categorical,5
Boolean,1
Date,0
Text (Unique),0
Rejected,0
